In [1]:
import os
import sys
import numpy as np
import h5py
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork
from data.CleanSoundsDataset import PsychophysicsCleanSoundsDataset

# Specify Network to train
TODO: This should be converted to a script that accepts arguments for which network to train

In [2]:
from pbranchednetwork_a1 import PBranchedNetwork_A1SeparateHP
PNetClass = PBranchedNetwork_A1SeparateHP
pnet_name = 'a1v2'

In [3]:
from pbranchednetwork_conv1 import PBranchedNetwork_Conv1SeparateHP
PNetClass = PBranchedNetwork_Conv1SeparateHP
pnet_name = 'conv1v2'

In [2]:
from pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
PNetClass = PBranchedNetwork_AllSeparateHP
pnet_name = 'allv2'

# Parameters

In [3]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')
BATCH_SIZE = 50 #64
NUM_WORKERS = 2
PIN_MEMORY = True
NUM_EPOCHS = 50

lr = 1E-4
engram_dir = '/mnt/smb/locker/issa-locker/users/Erica/'
checkpoints_dir = f'{engram_dir}hcnn/checkpoints/'
tensorboard_dir = f'{engram_dir}hcnn/tensorboard/'

Device: cuda:0


In [4]:
!nvidia-smi

Tue Jun 14 00:23:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 455.45.01    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:1D:00.0 Off |                  N/A |
| 27%   29C    P8    13W / 250W |      3MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:1F:00.0 Off |                  N/A |
| 27%   

# Load network and optimizer

In [5]:
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

/share/issa/users/es3773/hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/issa/users/es3773/hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


<All keys matched successfully>

In [6]:
pnet = PNetClass(net, build_graph=True)

In [7]:
pnet.eval()

PBranchedNetwork_A1SeparateHP(
  (backbone): BranchedNetwork(
    (speech_branch): Sequential(
      (conv1): ConvLayer(
        (block): Sequential(
          (0): Conv2d(1, 96, kernel_size=(6, 14), stride=(3, 3), padding=(2, 6))
          (1): ReLU()
        )
      )
      (rnorm1): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool1): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv2): ConvLayer(
        (block): Sequential(
          (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(1, 2))
          (1): ReLU()
        )
      )
      (rnorm2): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool2): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv3): ConvLayer(
        (block): Sequential(
          (0): Conv2d(256, 512, kernel_size=(

In [8]:
pnet.to(DEVICE)
optimizer = torch.optim.Adam(
    [{'params':getattr(pnet,f"pcoder{x+1}").pmodule.parameters(), 'lr':lr} for x in range(pnet.number_of_pcoders)],
    weight_decay=5e-4)

# Load PsychoPhysics Dataset

In [9]:
f_in = h5py.File(f"{engram_dir}PsychophysicsWord2017W_not_resampled.hdf5", 'r')

In [ ]:
f_metadata = np.load(f"{engram_dir}PsychophysicsWord2017W_999c6fc475be1e82e114ab9865aa5459e4fd329d.__META.npy", 'r')

In [ ]:
f_key = np.load(f"{engram_dir}PsychophysicsWord2017W_999c6fc475be1e82e114ab9865aa5459e4fd329d.__META_key.npy", 'r')

In [ ]:
def getPsychophysics2017WCleanCochleagrams():
    
    cochleagrams_clean = []
   
    cochleagrams = []
    for batch_ii in range(0,15300,100):
        hdf5_path = '/mnt/smb/locker/issa-locker/users/Erica/cgrams_for_noise_robustness_analysis/PsychophysicsWord2017W_clean/batch_'+str(batch_ii)+'_to_'+str(batch_ii+100)+'.hdf5'
        with h5py.File(hdf5_path, 'r') as f_in:
            cochleagrams += list(f_in['data'])

    return cochleagrams
clean_in = getPsychophysics2017WCleanCochleagrams()
clean_in = np.array(clean_in)

In [ ]:
labels = []
for word in f_metadata['word']:
    idx = np.argwhere(f_key == word)
    if len(idx) == 0:
        labels.append(-1)
    else:
        labels.append(idx.item())
labels = np.array(labels)
labels += 1

In [ ]:
orig_dset = []
for _orig_dset in f_metadata['orig_dset']:
    _orig_dset = str(_orig_dset, 'utf-8')
    _orig_dset = 'WSJ' if 'WSJ' in _orig_dset else 'Timit'
    orig_dset.append(_orig_dset)
orig_dset = np.array(orig_dset)

# Set up dataset

In [ ]:
full_dataset = PsychophysicsCleanSoundsDataset(
    clean_in, labels, orig_dset, exclude_timit=True
    )
n_train = int(len(full_dataset)*0.9)
train_dataset = Subset(full_dataset, np.arange(n_train))
eval_dataset = Subset(full_dataset, np.arange(n_train, len(full_dataset)))
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )
eval_loader = DataLoader(
    eval_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )

# Set up checkpoints and tensorboards

In [ ]:
checkpoint_path = os.path.join(checkpoints_dir, f"{pnet_name}")
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, pnet_name + '-{epoch}-{type}.pth')

# summarywriter
from torch.utils.tensorboard import SummaryWriter
tensorboard_path = os.path.join(tensorboard_dir, f"{pnet_name}")
if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)
sumwriter = SummaryWriter(tensorboard_path, filename_suffix=f'')

# Train

In [ ]:
loss_function = torch.nn.MSELoss()
for epoch in range(1, NUM_EPOCHS+1):
    train_pcoders(pnet, optimizer, loss_function, epoch, train_loader, DEVICE, sumwriter)
    eval_pcoders(pnet, loss_function, epoch, eval_loader, DEVICE, sumwriter)

    # save checkpoints every 5 epochs
    if epoch % 5 == 0:
        torch.save(pnet.state_dict(), checkpoint_path.format(epoch=epoch, type='regular'))

/home/es3773/.local/lib/python3.7/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Training Epoch: 1 [50/13219]	Loss: 48283.1875
Training Epoch: 1 [100/13219]	Loss: 26204.1973
Training Epoch: 1 [150/13219]	Loss: 17961.8164
Training Epoch: 1 [200/13219]	Loss: 20394.4180
Training Epoch: 1 [250/13219]	Loss: 26420.9844
Training Epoch: 1 [300/13219]	Loss: 26866.3906
Training Epoch: 1 [350/13219]	Loss: 24480.7754
Training Epoch: 1 [400/13219]	Loss: 19651.7129
Training Epoch: 1 [450/13219]	Loss: 15681.5283
Training Epoch: 1 [500/13219]	Loss: 14367.2773
Training Epoch: 1 [550/13219]	Loss: 15406.2051
Training Epoch: 1 [600/13219]	Loss: 15659.7402
Training Epoch: 1 [650/13219]	Loss: 16077.7100
Training Epoch: 1 [700/13219]	Loss: 16421.9375
Training Epoch: 1 [750/13219]	Loss: 15502.5732
Training Epoch: 1 [800/13219]	Loss: 13262.4473
Training Epoch: 1 [850/13219]	Loss: 11620.0312
Training Epoch: 1 [900/13219]	Loss: 11201.5186
Training Epoch: 1 [950/13219]	Loss: 10886.3682
Training Epoch: 1 [1000/13219]	Loss: 12153.4697
Training Epoch: 1 [1050/13219]	Loss: 11770.5449
Training Epo

Training Epoch: 1 [8800/13219]	Loss: 3054.5881
Training Epoch: 1 [8850/13219]	Loss: 3074.7666
Training Epoch: 1 [8900/13219]	Loss: 3153.3689
Training Epoch: 1 [8950/13219]	Loss: 3068.8169
Training Epoch: 1 [9000/13219]	Loss: 3028.8743
Training Epoch: 1 [9050/13219]	Loss: 3116.4746
Training Epoch: 1 [9100/13219]	Loss: 3111.8782
Training Epoch: 1 [9150/13219]	Loss: 3093.8135
Training Epoch: 1 [9200/13219]	Loss: 3066.3623
Training Epoch: 1 [9250/13219]	Loss: 3158.4348
Training Epoch: 1 [9300/13219]	Loss: 3044.4260
Training Epoch: 1 [9350/13219]	Loss: 3051.1345
Training Epoch: 1 [9400/13219]	Loss: 3076.4529
Training Epoch: 1 [9450/13219]	Loss: 2954.8838
Training Epoch: 1 [9500/13219]	Loss: 2833.8411
Training Epoch: 1 [9550/13219]	Loss: 2857.6765
Training Epoch: 1 [9600/13219]	Loss: 2857.7402
Training Epoch: 1 [9650/13219]	Loss: 2975.7339
Training Epoch: 1 [9700/13219]	Loss: 2908.2051
Training Epoch: 1 [9750/13219]	Loss: 2993.5847
Training Epoch: 1 [9800/13219]	Loss: 2915.6770
Training Epoc

Training Epoch: 2 [4200/13219]	Loss: 1929.2828
Training Epoch: 2 [4250/13219]	Loss: 2006.1085
Training Epoch: 2 [4300/13219]	Loss: 2015.4489
Training Epoch: 2 [4350/13219]	Loss: 1900.1544
Training Epoch: 2 [4400/13219]	Loss: 1983.4686
Training Epoch: 2 [4450/13219]	Loss: 1910.0205
Training Epoch: 2 [4500/13219]	Loss: 1821.3707
Training Epoch: 2 [4550/13219]	Loss: 1984.9662
Training Epoch: 2 [4600/13219]	Loss: 1891.2726
Training Epoch: 2 [4650/13219]	Loss: 1963.4421
Training Epoch: 2 [4700/13219]	Loss: 1879.2384
Training Epoch: 2 [4750/13219]	Loss: 1977.6809
Training Epoch: 2 [4800/13219]	Loss: 1924.3112
Training Epoch: 2 [4850/13219]	Loss: 1900.6223
Training Epoch: 2 [4900/13219]	Loss: 1963.2020
Training Epoch: 2 [4950/13219]	Loss: 1912.2756
Training Epoch: 2 [5000/13219]	Loss: 1853.3003
Training Epoch: 2 [5050/13219]	Loss: 1904.2053
Training Epoch: 2 [5100/13219]	Loss: 1902.0833
Training Epoch: 2 [5150/13219]	Loss: 1924.4987
Training Epoch: 2 [5200/13219]	Loss: 1830.1880
Training Epoc

Training Epoch: 2 [12900/13219]	Loss: 1371.5718
Training Epoch: 2 [12950/13219]	Loss: 1547.6409
Training Epoch: 2 [13000/13219]	Loss: 1494.9054
Training Epoch: 2 [13050/13219]	Loss: 1475.3027
Training Epoch: 2 [13100/13219]	Loss: 1484.1483
Training Epoch: 2 [13150/13219]	Loss: 1489.2255
Training Epoch: 2 [13200/13219]	Loss: 1427.2339
Training Epoch: 2 [13219/13219]	Loss: 1509.0775
Training Epoch: 2 [1469/1469]	Loss: 1455.6856
Training Epoch: 3 [50/13219]	Loss: 1520.6245
Training Epoch: 3 [100/13219]	Loss: 1507.2808
Training Epoch: 3 [150/13219]	Loss: 1441.5325
Training Epoch: 3 [200/13219]	Loss: 1520.8237
Training Epoch: 3 [250/13219]	Loss: 1442.8201
Training Epoch: 3 [300/13219]	Loss: 1422.6147
Training Epoch: 3 [350/13219]	Loss: 1429.6383
Training Epoch: 3 [400/13219]	Loss: 1396.1559
Training Epoch: 3 [450/13219]	Loss: 1426.1342
Training Epoch: 3 [500/13219]	Loss: 1450.2825
Training Epoch: 3 [550/13219]	Loss: 1424.5626
Training Epoch: 3 [600/13219]	Loss: 1361.8462
Training Epoch: 3 [

...

Training Epoch: 14 [11600/13219]	Loss: 700.8413
Training Epoch: 14 [11650/13219]	Loss: 754.0212
Training Epoch: 14 [11700/13219]	Loss: 716.5876
Training Epoch: 14 [11750/13219]	Loss: 722.1716
Training Epoch: 14 [11800/13219]	Loss: 727.4462
Training Epoch: 14 [11850/13219]	Loss: 738.7188
Training Epoch: 14 [11900/13219]	Loss: 742.1146
Training Epoch: 14 [11950/13219]	Loss: 728.0741
Training Epoch: 14 [12000/13219]	Loss: 716.2756
Training Epoch: 14 [12050/13219]	Loss: 706.9046
Training Epoch: 14 [12100/13219]	Loss: 716.1285
Training Epoch: 14 [12150/13219]	Loss: 749.6113
Training Epoch: 14 [12200/13219]	Loss: 726.3292
Training Epoch: 14 [12250/13219]	Loss: 776.8123
Training Epoch: 14 [12300/13219]	Loss: 767.0355
Training Epoch: 14 [12350/13219]	Loss: 722.6252
Training Epoch: 14 [12400/13219]	Loss: 722.2745
Training Epoch: 14 [12450/13219]	Loss: 765.0991
Training Epoch: 14 [12500/13219]	Loss: 729.5065
Training Epoch: 14 [12550/13219]	Loss: 724.6375
Training Epoch: 14 [12600/13219]	Loss: 7

Training Epoch: 15 [7050/13219]	Loss: 731.7645
Training Epoch: 15 [7100/13219]	Loss: 706.7906
Training Epoch: 15 [7150/13219]	Loss: 727.5759
Training Epoch: 15 [7200/13219]	Loss: 687.4910
Training Epoch: 15 [7250/13219]	Loss: 727.7625
Training Epoch: 15 [7300/13219]	Loss: 718.5217
Training Epoch: 15 [7350/13219]	Loss: 718.4708
Training Epoch: 15 [7400/13219]	Loss: 753.5355
Training Epoch: 15 [7450/13219]	Loss: 714.9842
Training Epoch: 15 [7500/13219]	Loss: 722.2338
Training Epoch: 15 [7550/13219]	Loss: 712.0779
Training Epoch: 15 [7600/13219]	Loss: 699.7518
Training Epoch: 15 [7650/13219]	Loss: 683.3535
Training Epoch: 15 [7700/13219]	Loss: 697.5016
Training Epoch: 15 [7750/13219]	Loss: 721.4477
Training Epoch: 15 [7800/13219]	Loss: 696.9599
Training Epoch: 15 [7850/13219]	Loss: 698.7310
Training Epoch: 15 [7900/13219]	Loss: 712.4798
Training Epoch: 15 [7950/13219]	Loss: 708.3447
Training Epoch: 15 [8000/13219]	Loss: 712.8180
Training Epoch: 15 [8050/13219]	Loss: 761.0558
Training Epoc

Training Epoch: 16 [2450/13219]	Loss: 727.6563
Training Epoch: 16 [2500/13219]	Loss: 753.9682
Training Epoch: 16 [2550/13219]	Loss: 763.3097
Training Epoch: 16 [2600/13219]	Loss: 725.5445
Training Epoch: 16 [2650/13219]	Loss: 790.8953
Training Epoch: 16 [2700/13219]	Loss: 761.7496
Training Epoch: 16 [2750/13219]	Loss: 715.7839
Training Epoch: 16 [2800/13219]	Loss: 686.4170
Training Epoch: 16 [2850/13219]	Loss: 708.5341
Training Epoch: 16 [2900/13219]	Loss: 695.4252
Training Epoch: 16 [2950/13219]	Loss: 724.5978
Training Epoch: 16 [3000/13219]	Loss: 748.6279
Training Epoch: 16 [3050/13219]	Loss: 740.8932
Training Epoch: 16 [3100/13219]	Loss: 699.7446
Training Epoch: 16 [3150/13219]	Loss: 717.2307
Training Epoch: 16 [3200/13219]	Loss: 718.6259
Training Epoch: 16 [3250/13219]	Loss: 692.0477
Training Epoch: 16 [3300/13219]	Loss: 736.0128
Training Epoch: 16 [3350/13219]	Loss: 715.4318
Training Epoch: 16 [3400/13219]	Loss: 703.3478
Training Epoch: 16 [3450/13219]	Loss: 714.9396
Training Epoc

Training Epoch: 16 [11150/13219]	Loss: 712.3098
Training Epoch: 16 [11200/13219]	Loss: 704.4591
Training Epoch: 16 [11250/13219]	Loss: 686.9704
Training Epoch: 16 [11300/13219]	Loss: 685.7860
Training Epoch: 16 [11350/13219]	Loss: 694.5526
Training Epoch: 16 [11400/13219]	Loss: 680.9238
Training Epoch: 16 [11450/13219]	Loss: 686.9296
Training Epoch: 16 [11500/13219]	Loss: 738.8110
Training Epoch: 16 [11550/13219]	Loss: 719.6867
Training Epoch: 16 [11600/13219]	Loss: 707.3965
Training Epoch: 16 [11650/13219]	Loss: 718.5328
Training Epoch: 16 [11700/13219]	Loss: 682.1667
Training Epoch: 16 [11750/13219]	Loss: 720.3328
Training Epoch: 16 [11800/13219]	Loss: 669.2908
Training Epoch: 16 [11850/13219]	Loss: 667.8064
Training Epoch: 16 [11900/13219]	Loss: 711.2332
Training Epoch: 16 [11950/13219]	Loss: 701.3683
Training Epoch: 16 [12000/13219]	Loss: 680.5057
Training Epoch: 16 [12050/13219]	Loss: 711.0894
Training Epoch: 16 [12100/13219]	Loss: 663.7670
Training Epoch: 16 [12150/13219]	Loss: 6

Training Epoch: 17 [6550/13219]	Loss: 701.6326
Training Epoch: 17 [6600/13219]	Loss: 673.6462
Training Epoch: 17 [6650/13219]	Loss: 695.2227
Training Epoch: 17 [6700/13219]	Loss: 684.0609
Training Epoch: 17 [6750/13219]	Loss: 715.0433
Training Epoch: 17 [6800/13219]	Loss: 685.8969
Training Epoch: 17 [6850/13219]	Loss: 723.5406
Training Epoch: 17 [6900/13219]	Loss: 695.9122
Training Epoch: 17 [6950/13219]	Loss: 667.4089
Training Epoch: 17 [7000/13219]	Loss: 691.5577
Training Epoch: 17 [7050/13219]	Loss: 705.8397
Training Epoch: 17 [7100/13219]	Loss: 701.1441
Training Epoch: 17 [7150/13219]	Loss: 692.4494
Training Epoch: 17 [7200/13219]	Loss: 727.1346
Training Epoch: 17 [7250/13219]	Loss: 703.5995
Training Epoch: 17 [7300/13219]	Loss: 699.8321
Training Epoch: 17 [7350/13219]	Loss: 710.7709
Training Epoch: 17 [7400/13219]	Loss: 687.4626
Training Epoch: 17 [7450/13219]	Loss: 728.2179
Training Epoch: 17 [7500/13219]	Loss: 754.9102
Training Epoch: 17 [7550/13219]	Loss: 694.9301
Training Epoc